In [1]:
import rioxarray 
import geopandas
import xarray as xr
from shapely.geometry import mapping

In [2]:
diri = "/Users/fadhlilrizki/Documents/PhD/thesis/data/"
df_pr = xr.open_dataset(diri+"agcd_v1_precip_calib_r025_daily_1981-2020.unmasked.shift1day.nc")
pr = df_pr.precip
geodf = geopandas.read_file(diri+"shpAUS/"+"AUS_2021_AUST_GDA2020.shp", crs="epsg:4326")
geodf = geodf[geodf['AUS_CODE21'] == 'AUS']
pr

<xarray.DataArray 'precip' (time: 14396, lat: 138, lon: 177)>
[351636696 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1980-12-31T09:00:00 ... 2020-05-30T09:00:00
  * lon      (lon) float32 112.0 112.2 112.5 112.8 ... 155.2 155.5 155.8 156.0
  * lat      (lat) float32 -44.5 -44.25 -44.0 -43.75 ... -10.75 -10.5 -10.25
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6600
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement

In [3]:
geodf

,AUS_CODE21,AUS_NAME21,CHG_FLAG21,CHG_LBL21,AREASQKM21,LOCI_URI21,geometry
0,AUS,Australia,0,No change,7.688095e+06,http://linked.data.gov.au/dataset/asgsed3/AUS/AUS,"MULTIPOLYGON (((96.91524 -12.15313, 96.91523 -..."


In [4]:
pr = pr.sel(time=slice('1981-01-01','2020-05-30'))
pr

<xarray.DataArray 'precip' (time: 14395, lat: 138, lon: 177)>
[351612270 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01T09:00:00 ... 2020-05-30T09:00:00
  * lon      (lon) float32 112.0 112.2 112.5 112.8 ... 155.2 155.5 155.8 156.0
  * lat      (lat) float32 -44.5 -44.25 -44.0 -43.75 ... -10.75 -10.5 -10.25
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6600
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement

In [5]:
pr.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
pr.rio.write_crs("epsg:4326", inplace=True)
clipped = pr.rio.clip(geodf.geometry.apply(mapping), geodf.crs, drop=True)

In [6]:
# clipped.to_netcdf(diri+"agcd_v1_precip_calib_r025_daily_1981-2020.maskedshp.nc")

In [7]:
eastcoast_shp = geopandas.read_file(diri+"shpAUS/"+"eastcoast_upper.shp", crs="epsg:4326")
clip_eastcoast = clipped.rio.clip(eastcoast_shp.geometry.apply(mapping), eastcoast_shp.crs, drop=False)

In [8]:
! rm {diri}"agcd_eastcoast.shift1day.nc"

In [9]:
clip_eastcoast

<xarray.DataArray 'precip' (time: 14395, lat: 132, lon: 163)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 1981-01-01T09:00:00 ... 2020-05-30T09:...
  * lon          (lon) float32 113.0 113.2 113.5 113.8 ... 153.0 153.2 153.5
  * lat          (lat) float32 -43.5 -43.25 -43.0 -42.75 ... -11.25 -11.0 -10.75
    spatial_ref  int64 0
Attributes:
    standard_name:                 lwe_thickness_of_precipitation_amount
    long_name:                     Daily precipitation
    units:                         mm
    cell_methods:                  time: sum
    analysis_version_number:       3.01
    number_of_stations_reporting:  6600
    frequency:                     daily
    length_scale_for_analysis:     80.0
    coverage_content_type:         physicalMeasurement

In [10]:
clip_eastcoast.to_netcdf(diri+"agcd_eastcoast.shift1day.nc")